1. ResNet50: 224x224
2. VGG16: 224x224
3. Xception: 299x299
4. Inception: 299x299

# Image Augmentation(증강) with split-folders

In [ ]:
import os

import cv2
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import pickle
from tqdm.notebook import tqdm

import splitfolders

In [2]:
folder_path = 'img'
label_names = os.listdir(folder_path)
label_names

['1', '2', '3', '4', '5']

In [3]:
os.mkdir('resize')
for label in label_names:
    dir_path = 'resize/'+ label
    os.mkdir(dir_path)

In [4]:
# 카테고리별로 이미지파일 경로 리스트 사전 만들기
# keys : 카테고리명
# values : 리스트 of 이미지 경로들
dataset = {} # 카테고리명 : [이미지1경로, 이미지2경로, ...]

for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'1': ['img/1/0000121.jpg',
  'img/1/0000124.jpg',
  'img/1/0000182.jpg',
  'img/1/0000190.jpg',
  'img/1/0000249.jpg',
  'img/1/0000255.jpg',
  'img/1/0000257.jpg',
  'img/1/0000258.jpg',
  'img/1/0000259.jpg',
  'img/1/0000260.jpg',
  'img/1/0000261.jpg',
  'img/1/0000263.jpg',
  'img/1/0000339.jpg',
  'img/1/0000458.jpg',
  'img/1/0000459.jpg',
  'img/1/0000460.jpg',
  'img/1/0000494.jpg',
  'img/1/0000495.jpg',
  'img/1/0000497.jpg',
  'img/1/0000505.jpg',
  'img/1/0000506.jpg',
  'img/1/0000507.jpg',
  'img/1/0000510.jpg',
  'img/1/0000512.jpg',
  'img/1/0000515.jpg',
  'img/1/0000517.jpg',
  'img/1/0000519.jpg',
  'img/1/0000520.jpg',
  'img/1/0000522.jpg',
  'img/1/0000525.jpg',
  'img/1/0000526.jpg',
  'img/1/0000527.jpg',
  'img/1/0000528.jpg',
  'img/1/0000556.jpg',
  'img/1/0000559.jpg',
  'img/1/0000589.jpg',
  'img/1/0000590.jpg',
  'img/1/0000638.jpg',
  'img/1/0000783.jpg',
  'img/1/0000807.jpg',
  'img/1/0000813.jpg',
  'img/1/0000818.jpg',
  'img/1/0000836.jpg',
  'img

## Resize with padding (ResNet50,VGG16: 224x224)

In [5]:
label2index = {'아주낮음' : 0, '낮음' : 1 , '보통' : 2 , '높음' : 3,
               '아주높음' : 4}

In [6]:
def resize_img(img_path, img_size=224):
    img = cv2.imread(img_path)

    if(img.shape[1] > img.shape[0]) :
        ratio = img_size/img.shape[1]
    else :
        ratio = img_size/img.shape[0]

    img = cv2.resize(img, dsize=(0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_LINEAR)

    # 그림 주변에 검은색으로 칠하기
    w, h = img.shape[1], img.shape[0]

    dw = (img_size-w)/2 # img_size와 w의 차이
    dh = (img_size-h)/2 # img_size와 h의 차이

    M = np.float32([[1,0,dw], [0,1,dh]])  #(2*3 이차원 행렬)
    img_re = cv2.warpAffine(img, M, (224, 224)) #이동변환
    cv2.imwrite('resize/{0}/{1}'.format(label, img_path.split("/")[-1]) , img_re)

In [7]:
# 이미지 리사이즈
# 1. 이미지 하나씩 읽어들여서
# 2. 리사이즈한다음
# 3. images, labels 생성
for label, img_paths in dataset.items():
    for img_path in img_paths:
        resize_img(img_path, img_size=224)

## Resize with padding2 (Xception,Inception: 299x299)

In [ ]:
# def resize_img(img_path, img_size=299):
#   img = cv2.imread(img_path)

#   if (img.shape[1] > img.shape[0]):
#     ratio = img_size / img.shape[1]
#   else:
#     ratio = img_size / img.shape[0]

#   img = cv2.resize(img, dsize=(0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_LINEAR)

#   # 그림 주변에 검은색으로 칠하기
#   w, h = img.shape[1], img.shape[0]

#   dw = (img_size - w) / 2  # img_size와 w의 차이
#   dh = (img_size - h) / 2  # img_size와 h의 차이

#   M = np.float32([[1, 0, dw], [0, 1, dh]])  # (2*3 이차원 행렬)
#   img_re = cv2.warpAffine(img, M, (299, 299))  # 이동변환
#   cv2.imwrite('/content/resize/{0}/{1}'.format(label, img_path.split('/')[-1]), img_re)

In [ ]:
# for label, img_paths in dataset.items():
#     for img_path in img_paths:
#         resize_img(img_path, img_size=299)

## Split Dataset

In [8]:
splitfolders.ratio('resize', output='dataset', seed=77, ratio=(0.6, 0.2, 0.2))

Copying files: 116507 files [09:00, 215.69 files/s]
